In [1]:
import re
import requests
from bs4 import BeautifulSoup as bs
from requests.exceptions import RequestException

In [2]:
url = 'https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
}
html = requests.get( url , headers = headers)
if html.status_code==200:
        #判断请求是否成功
    print('succeeded')
else:
    print('wrong')

succeeded


In [3]:
soup = bs(html.text, 'html.parser')

In [6]:
class_movie = 'ipc-metadata-list-summary-item sc-10233bc-0 iherUv cli-parent'
class_movie = ['li', class_movie]
class_title = 'ipc-title__text'
class_title = ['h3', class_title]
class_detail = 'sc-b189961a-8 kLaxqf cli-title-metadata-item'
class_detail = ['span', class_detail]
class_rating = 'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'
class_rating = ['span', class_rating]
class_link = 'ipc-title-link-wrapper'
class_link = ['a', class_link]


movies = soup.find_all(class_movie[0], {'class': class_movie[1]})

info_list = []
for each in movies:
    each_info = []

    title = each.find(class_title[0], class_ = class_title[1]).string
    index = title.find('. ')+2
    each_info += [title[index:]]

    details = each.find_all(class_detail[0], class_ = class_detail[1])
    for d in details:
        each_info += [d.string]
    rating = each.find(class_rating[0], class_ = class_rating[1]).attrs['aria-label']
    each_info += [rating[-3:]]
    link = each.find(class_link[0], class_ = class_link).attrs['href']
    index1 = link.find('/tt')
    index2 = link.find('/?')
    link = link[index1+1:index2]
    each_info += [link]

    info_list.append(each_info)
    
    #返回节点中评分的属性
    #print(title, rating, details)
    #info = info + [[title, detail, rating]]
    
print(info_list[0])
#排名页爬完了
#https://www.imdb.com/title/tt0903747/reviews/?ref_=tt_ov_rt
#转换成评论页面链接

['Breaking Bad', '2008–2013', '62 eps', 'MA15+', '9.5', 'tt0903747']


In [14]:
for i in info_list:
    if len(i) < 6:
        i.insert(3, 'None')

In [15]:
import pandas as pd
t250 = pd.DataFrame(info_list, columns=['title', 'year', 'eps', 'age_res', 'rating', 'link'])

eps = t250['eps']
eps_new = []
for i in range(0,250):
    eps_new.append(eps[i][:-4])
t250['eps'] = eps_new

print(t250.head())
t250.to_csv('t250.csv')

              title       year eps age_res rating       link
0      Breaking Bad  2008–2013  62   MA15+    9.5  tt0903747
1   Planet Earth II       2016   6      PG    9.5  tt5491994
2      Planet Earth       2006  11      PG    9.4  tt0795176
3  Band of Brothers       2001  10       M    9.4  tt0185906
4         Chernobyl       2019   5   MA15+    9.3  tt7366338
